In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)
import warnings
warnings.filterwarnings('ignore')


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
# enabling the pretrained model for trainig our custom model using tensorflow hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

# creating a method for embedding and will using method for every input layer 
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [23]:

    DROPOUT = 0.1
    # Taking the question1 as input and ceating a embedding for each question before feed it to neural network
    q1 = layers.Input(shape=(1,), dtype=tf.string)
    embedding_q1 = layers.Lambda(UniversalEmbedding, output_shape=(512,))(q1)
    # Taking the question2 and doing the same thing mentioned above, using the lambda function
    q2 = layers.Input(shape=(1,), dtype=tf.string)
    embedding_q2 = layers.Lambda(UniversalEmbedding, output_shape=(512,))(q2)

    # Concatenating the both input layer
    merged = layers.concatenate([embedding_q1, embedding_q2])
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    # Normalizing the input layer,applying dense and dropout  layer for fully connected model and to avoid overfitting 
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    merged = layers.BatchNormalization()(merged)
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    merged = layers.BatchNormalization()(merged)
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    # Using the Sigmoid as the activation function and binary crossentropy for binary classifcation as 0 or 1
    merged = layers.BatchNormalization()(merged)
    pred = layers.Dense(2, activation='sigmoid')(merged)
    model = Model(inputs=[q1,q2], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 512)          0           input_5[0][0]                    
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 512)          0           input_6[0][0]                    
____________________________________________________________________________________________

In [5]:
# while executing below code 2nd time onwards it display a "session closed" warning message , please do not consider that, 
# still it produce the expected result as we already tranied the model and stored it in local

In [51]:
  q1 = input("Type Question 1 here -->")
  q2 = input("Type Question 2 here -->")
  print("Question 1 --> " + q1) 
  print("Question 2 --> " + q2)
  q1 = np.array([[q1],[q1]])
  q2 = np.array([[q2],[q2]])


# Using the same tensorflow session for embedding the test string
  with tf.Session() as session:
    K.set_session(tf.Session())
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    # Loading the save weights
    model.load_weights('quora-model-10-0.84.hdf5')  
    # Predicting the similarity between the two input questions 
    predicts = model.predict([q1, q2], verbose=0)
    predict_logits = predicts.argmax(axis=1)
    print("----FINAL RESULT----")
    if(predict_logits[0] == 1):
      print("****Questions are Similar****")
    else:
      print("****Questions are not Similar****")

Type Question 1 here -->Is it possible to uninstall the integrated Siri software on the iPhone 4S?
Type Question 2 here -->Is Siri a part of the iPhone 4S?
Question 1 --> Is it possible to uninstall the integrated Siri software on the iPhone 4S?
Question 2 --> Is Siri a part of the iPhone 4S?
----FINAL RESULT----
****Questions are not Similar****


# Test Questions

In [42]:
'''
Q1 = "How does weight loss transformation affect your personality ?", Q2 = "Did your weight loss transformation affect your personality ?"
Q1 = "Can excessive amounts of Vitamin C cause me to have a miscarriage?",Q2 = "How can Vitamin C cause a miscarriage?"
Q1 = "Which are the best engineering fields?",Q2 = "What is the best field of engineering?"
Q1 = "How do you get deleted Instagram chats?", Q2 = "How can I view deleted Instagram dms?
Q1 = "Is it possible to uninstall the integrated Siri software on the iPhone 4S?", Q2= "Is Siri a part of the iPhone 4S?"
Q1 = "What are T-rays?", Q2 = "What is an X-ray?"


Q1 = Can excessive amounts of Vitamin C cause me to have a miscarriage?",Q2 = "How can Vitamin C cause a miscarriage?" 1
Q1 ="Which are the best engineering fields?",Q2 = "What is the best field of engineering?","1"
Q1 ="How do you get deleted Instagram chats?",Q2 = "How can I view deleted Instagram dms?","1"
Q1 ="What if I hired two private eyes and ordered them to follow each other?",Q2 = "Would I be able to hire two private investigators and then get them to follow each other?","1"
Q1 ="What is the best combination of courses I can take up along with CA to enhance my career?",Q2 = "What courses must be taken along with CA course?","1"
Q1 ="What are some must watch TV shows before you die?",Q2 = "Are there any must watch TV shows?","1"
Q1 ="I am ugly and fat, how to lose weight?",Q2 = "How do I actually lose weight?","1"
Q1 ="How can I clear the bank exams after degree?",Q2 = "How can I clear bank exams after my graduation? Which books to refer?","1"
Q1 ="Setting aside religious teachings, what do you consider as possible evidence for life after death?",Q2 = "Is there any tangible evidence for life after death?","1"



Q1 ="Is it possible to uninstall the integrated Siri software on the iPhone 4S?",Q2 = "Is Siri a part of the iPhone 4S?","0"
Q1 ="Is Uber safe to use if you're a female riding solo?",Q2 = "Is it safe to use Uber in Dubai?","0"
Q1 ="What is the nearest airport to Daytona Beach, and how do this cities tourist attractions compare to Billing's?",Q2 = "What is the nearest airport to Daytona Beach, and how do this cities tourist attractions compare to Orlando's?","0"
'''




'\nQ1 = "How does weight loss transformation affect your personality ?", Q2 = "Did your weight loss transformation affect your personality ?"\nQ1 = "Can excessive amounts of Vitamin C cause me to have a miscarriage?",Q2 = "How can Vitamin C cause a miscarriage?"\nQ1 = "Which are the best engineering fields?",Q2 = "What is the best field of engineering?"\nQ1 = "How do you get deleted Instagram chats?", Q2 = "How can I view deleted Instagram dms?\nQ1 = "Is it possible to uninstall the integrated Siri software on the iPhone 4S?", Q2= "Is Siri a part of the iPhone 4S?"\nQ1 = "What are T-rays?", Q2 = "What is an X-ray?"\n'